In [10]:
#Import the required libraries
import os
import cv2
import numpy as np
import pandas as pd
import pytesseract

#Natural Language Processing libraries
from spacy.matcher import PhraseMatcher
import nltk
import re as re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

In [11]:
# Importing tesseract from the location
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe" 

In [16]:
#Testing on one image
img = cv2.imread('./Dataset/Test957.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

text = pytesseract.image_to_string(gray, lang='eng')

print(text)
cv2.imshow("gray",gray)
cv2.waitKey(0)
cv2.destroyWindow('gray')

Tea aha

We Me a mee | ee


In [4]:
import cv2
import os
import glob
img_dir = 'C:/Users/kanum/Desktop/Data_Science_Everyday/Hackathons/Love is Love/Dataset' # Enter Directory of all images 
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
data = []
filename = []
for f1 in files:
    file = os.path.basename(f1)
    filename.append(file)
    img = cv2.imread(f1)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) #converting to gray image
    gray_rm = cv2.medianBlur(gray,5) #removing noise
    text = pytesseract.image_to_string(gray_rm, lang='eng')
    data.append(text)

In [5]:
inp = pd.DataFrame()
inp['Filename'] = filename
inp['text'] = data
inp.head(5)

,Filename,text
0,Test100.jpg,
1,Test1001.jpg,ag\nti\nHe\n\nj\nif\nHy\n\n&
2,Test1012.jpg,
3,Test1022.jpg,LOVE\n©
4,Test103.jpg,1\n6\noO


I have extracted text from the images, we have performed OCR on images(I didn't preprocess the images yet). Now I can start performing Natural Language Processing on the extracted text.For extracting sentiment from the text I am implementing VADAR (Valence Aware Dictionary and sEntiment Reasoner)

__VADER provides a positive, negative, and neutral sentiment scalar, as well as a compound sentiment vector that combines the three__

In [9]:
analyzer = SentimentIntensityAnalyzer()

def clean_text(text):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', text).split())

inp["text"] = inp['text'].apply(lambda x: clean_text(x))
inp

def analyze_sentiment(text):
    analysis = TextBlob(text)
    
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity ==0:
        return 'Neutral'
    else:
        return 'Negative'

inp["Category"] = inp["text"].apply(lambda x: analyze_sentiment(x))

inp

,Filename,text,Category
0,Test100.jpg,,Neutral
1,Test1001.jpg,ag ti He j if Hy,Neutral
2,Test1012.jpg,,Neutral
3,Test1022.jpg,LOVE,Positive
4,Test103.jpg,1 6 oO,Neutral
...,...,...,...
234,Test945.jpg,Ty mas comsed a cor OF PeONLENeS Im Tes woaLn ...,Neutral
235,Test946.jpg,Lam not free while any woman is unfree even wh...,Positive
236,Test957.jpg,Le tT SO TN i ee ee,Neutral
237,Test979.jpg,,Neutral


In [7]:
submission = inp.drop(['text'],axis = 1)
submission = submission.replace(['old value'],'new value')
submission['Category'] = submission['Category'].replace(['Neutral'],'Random')
submission

,Filename,Category
0,Test100.jpg,Random
1,Test1001.jpg,Random
2,Test1012.jpg,Random
3,Test1022.jpg,Positive
4,Test103.jpg,Random
...,...,...
234,Test945.jpg,Random
235,Test946.jpg,Positive
236,Test957.jpg,Random
237,Test979.jpg,Random


In [8]:
submission.to_csv("submission2.csv",index = False)